In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.linear_model import LassoCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import statsmodels.formula.api as sm

In [ ]:
processed = '../../data/processed'
main_data = pd.read_pickle(Path(processed) / 'main_data.pkl')

In [ ]:
main_data.dtypes

In [ ]:
## Figure out what's up with people leaving before 3 PAs (identify early exit)

In [ ]:
PA_data = main_data.loc[main_data.spot <= 5, ['b_PA_in_G', 'spot', 'home', 'rating_pitcher_rgs']]
PA_data = PA_data.astype({
    'b_PA_in_G': 'int'
})

In [ ]:
# PA_data.b_PA_in_G.value_counts(normalize=True).sort_index()

In [ ]:
# PA_data[PA_data.home == 1].b_PA_in_G.value_counts(normalize=True).sort_index()

In [ ]:
# PA_data[(PA_data.home == 0)].b_PA_in_G.value_counts(normalize=True).sort_index()

In [ ]:
pd.crosstab(
    PA_data[(PA_data.home == 0)].b_PA_in_G, 
    PA_data[(PA_data.home == 0)].spot, 
    normalize = 'columns'
)

In [ ]:
pd.crosstab(
    PA_data[(PA_data.home == 1)].b_PA_in_G, 
    PA_data[(PA_data.home == 1)].spot, 
    normalize = 'columns'
)

In [ ]:
x_vars = [
    'spot', 'home'
]
preprocessor =  ColumnTransformer(
    [('spot', 'passthrough', x_vars)],
    remainder='drop'
)

clf = LassoCV(
    cv=5, random_state=0, max_iter=10000, n_jobs=-1,
)

fitted_model = Pipeline([
    ('select', preprocessor),
    ('scale', StandardScaler()),
    ('clf', clf),
])

fitted_model.fit(PA_data, PA_data['b_PA_in_G'].astype('float'))
PA_data['pred_HPPA'] = fitted_model.predict(PA_data)

In [ ]:
def adjust_plot(plt):
    plt.xaxis.grid(False)
    plt.yaxis.grid(True, linestyle='dashed', color='lightgrey')
    plt.set_axisbelow(True)
    plt.xaxis.set_ticks_position('none') 
    plt.yaxis.set_ticks_position('none') 

    plt.spines['left'].set_visible(False)
    plt.spines['right'].set_visible(False)
    plt.spines['top'].set_visible(False)

In [ ]:
plt1 = PA_data.b_PA_in_G.hist(rwidth = .9, bins = np.arange(.5, 12.5, 1))
adjust_plot(plt1)

In [ ]:
result = sm.ols(formula="b_PA_in_G ~ C(spot):C(home) + rating_pitcher_rgs", data=PA_data).fit()
print(result.summary())

In [ ]:
results = cross_val_score(fitted_model, PA_data, PA_data['b_PA_in_G'], cv=10, n_jobs=-1, scoring='r2')
print(results)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

In [ ]:
x = .3
y = 1 - (1- x)**(4/5)

p(H = 1)
p(H = 0)
p(pa = x)

Exp(H | p(H=1), p(pa=1), p(pa=2), p(pa=3), p(pa=4), p(pa=5))
